download fruit 360 Dataset from kaggle

In [ ]:
!pip install kaggle

Upload your account's API key to kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()  # سيطلب منك رفع الملف من جهازك

Saving kaggle.json to kaggle (1).json


setting up a key API

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

download dataset from kaggle

In [ ]:
!kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
 99% 953M/964M [00:11<00:00, 186MB/s]
100% 964M/964M [00:11<00:00, 87.0MB/s]


decompress data

In [ ]:
import zipfile
import os

# فك الضغط
with zipfile.ZipFile("fruits.zip", 'r') as zip_ref:
    zip_ref.extractall("fruits_dataset")

# التأكد من الملفات
os.listdir("fruits_dataset")

['fruits-360_dataset_100x100', 'fruits-360_dataset_original-size']

Verify contents fruits-360_dataset

In [ ]:
for root, dirs, files in os.walk('/content/fruits_dataset/fruits-360_dataset_original-size/'):
    print("Root:", root)
    print("Dirs:", dirs)
    print("Files:", files[:5])  # عرض أول 5 ملفات فقط
    break  # نكتفي بأول طبقة فقط

Root: /content/fruits_dataset/fruits-360_dataset_original-size/
Dirs: ['fruits-360-original-size']
Files: []


In [ ]:
import os
import os
print(os.listdir('/content/fruits_dataset/fruits-360_dataset_original-size/fruits-360-original-size/'))

['Test', 'readme.md', 'Training', 'Validation', 'Papers', 'Meta']


Preparing data

In [ ]:
import os

# تحديث المسارات
train_dir = '/content/fruits_dataset/fruits-360_dataset_original-size/fruits-360-original-size/Training'
validation_dir = '/content/fruits_dataset/fruits-360_dataset_original-size/fruits-360-original-size/Validation'
test_dir = '/content/fruits_dataset/fruits-360_dataset_original-size/fruits-360-original-size/Test'

# التأكد من وجود المسارات
print("Training Directory:", os.listdir(train_dir)[:5])  # عرض أول 5 مجلدات تدريب
print("Validation Directory:", os.listdir(validation_dir)[:5])  # عرض أول 5 مجلدات تحقق

Training Directory: ['apple_golden_2', 'pear_1', 'apple_6', 'apple_red_1', 'apple_golden_1']
Validation Directory: ['apple_golden_2', 'pear_1', 'apple_6', 'apple_red_1', 'apple_golden_1']


Training Model with data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# إعداد مُولد البيانات
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# تحميل بيانات التدريب
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

# تحميل بيانات التحقق
validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

# تحميل بيانات الاختبار
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

Found 6231 images belonging to 24 classes.
Found 3114 images belonging to 24 classes.
Found 3110 images belonging to 24 classes.


Model Building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# بناء النموذج
model = Sequential()

# إضافة طبقة Flatten لتحويل الصور إلى شكل مناسب للطبقات التالية
model.add(Flatten(input_shape=(28, 28, 3)))

# إضافة طبقة مخفية (Hidden Layer)
model.add(Dense(128, activation='relu'))

# إضافة طبقة إخراج (Output Layer) مع عدد الفئات
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

# تجميع النموذج
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# عرض ملخص للنموذج
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 2352)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         301,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 24)                  │           3,096 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 304,280 (1.16 MB)

 Trainable params: 304,280 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

Model Training

In [ ]:
# تدريب النموذج
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # يمكنك زيادة عدد epochs حسب الحاجة
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


194/194 ━━━━━━━━━━━━━━━━━━━━ 23s 109ms/step - accuracy: 0.4717 - loss: 1.8615 - val_accuracy: 0.9201 - val_loss: 0.4160
Epoch 2/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 201us/step - accuracy: 0.8750 - loss: 0.3868 - val_accuracy: 0.9000 - val_loss: 0.4615
Epoch 3/10


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


194/194 ━━━━━━━━━━━━━━━━━━━━ 40s 105ms/step - accuracy: 0.9128 - loss: 0.3573 - val_accuracy: 0.9910 - val_loss: 0.1561
Epoch 4/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 1.0000 - loss: 0.1227 - val_accuracy: 1.0000 - val_loss: 0.1456
Epoch 5/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 38s 110ms/step - accuracy: 0.9703 - loss: 0.1562 - val_accuracy: 0.9919 - val_loss: 0.0874
Epoch 6/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 1.0000 - loss: 0.0757 - val_accuracy: 1.0000 - val_loss: 0.0902
Epoch 7/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 38s 111ms/step - accuracy: 0.9874 - loss: 0.0915 - val_accuracy: 0.9739 - val_loss: 0.0876
Epoch 8/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 134us/step - accuracy: 0.9688 - loss: 0.0948 - val_accuracy: 1.0000 - val_loss: 0.0648
Epoch 9/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 40s 106ms/step - accuracy: 0.9964 - loss: 0.0468 - val_accuracy: 0.9997 - val_loss: 0.0288
Epoch 10/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 188us/step - accuracy: 1.0000 - loss: 0.0270 - val_accu

steps to save model

In [ ]:
# حفظ النموذج بتنسيق Keras الجديد
model.save('/content/my_model.keras')  # يمكنك تعديل المسار حسب المكان الذي تريده

Model Evaluation

In [ ]:
# تقييم النموذج على بيانات الاختبار
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))

# طباعة دقة الاختبار
print(f"Test Accuracy: {test_acc:.2%}")

# حفظ النموذج
model.save('/content/drive/MyDrive/models/my_model.h5')
print("Model saved successfully!")

Image Upload and Prediction

In [ ]:
!pip install ipywidgets

download important library

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 pydrive

Code to design an interactive interface to select an image from your device, and display the result

join Google Drive at Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/new_drive')

Access to the desired image

In [ ]:
!pip install gdown

In [ ]:
import gdown

# ID الخاص بالملف
file_id = "1OhZA4yOXb09fkNmWaM7cB7MgDbmJqzML"
# المسار الذي سيتم حفظ الملف فيه بعد التنزيل
output = "image.jpg"

# تحميل الملف
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)
print("تم تحميل الملف بنجاح!")

Image processing

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# تعريف أسماء الفئات (حسب التدريب)
class_names = ['تفاح', 'موز', 'برتقال', 'عنب', 'كيوي', 'ليمون', 'مانجو', 'بابايا', 'أناناس', 'فراولة', 'بطيخ', 'كمثرى', 'خوخ', 'كرز', 'رمان']
image_path = '/content/image.jpg'
# تحميل الصورة
img = load_img(image_path, target_size=(28, 28))  # تعديل أبعاد الصورة
img_array = img_to_array(img) / 255.0  # تطبيع الصورة
img_array = np.expand_dims(img_array, axis=0)  # إضافة بعد الباتش

# تحميل النموذج
# تحديد المسار إلى النموذج المدرب على Google Drive بتنسيق .keras
model = tf.keras.models.load_model('/content/my_model.keras')  # استبدل بالمسار الصحيح

show fruit name

In [ ]:
!pip install Pillow

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# تعريف أسماء الفئات (حسب التدريب)
class_names = [
    'apple_6', 'apple_braeburn_1', 'apple_crimson_snow_1', 'apple_golden_1',
    'apple_golden_2', 'apple_golden_3', 'apple_granny_smith_1', 'apple_hit_1',
    'apple_pink_lady_1', 'carrot_1', 'apple_red_2', 'apple_red_3', 'apple_red_delicios_1',
    'apple_red_yellow_1', 'apple_rotten_1', 'cabbage_white_1', 'appel_red', 'cucumber_1',
    'cucumber_3', 'eggplant_long_1', 'pear_1', 'pear_3', 'zucchini_1', 'zucchini_dark_1'
]
# تحميل الصورة باستخدام Pillow
image_path = '/content/image.jpg'  # ضع هنا مسار الصورة الخاصة بك
img = Image.open(image_path)  # فتح الصورة
img = img.resize((28, 28))  # تغيير حجم الصورة لتناسب النموذج
img = img.convert('RGB')  # التأكد من أن الصورة تحتوي على 3 قنوات فقط

# تحويل الصورة إلى مصفوفة وتطبيعها
img_array = np.array(img) / 255.0  # تطبيع الصورة
img_array = np.expand_dims(img_array, axis=0)  # إضافة بعد الباتش

# تحميل النموذج
model = tf.keras.models.load_model('/content/drive/MyDrive/models/my_model.h5')  # استبدل بالمسار الصحيح للنموذج

# التنبؤ باستخدام النموذج
predictions = model.predict(img_array)

# طباعة التنبؤات للتأكد من شكلها
print("التنبؤات: ", predictions)
print("عدد الفئات في النموذج: ", len(predictions[0]))

# التنبؤ بالفئة المتنبأ بها (اسم الفاكهة)
if len(predictions[0]) == len(class_names):
    predicted_class = class_names[np.argmax(predictions)]
    print(f"اسم الفاكهة المتنبأ بها: {predicted_class}")
else:
    print("عدد الفئات في النموذج لا يتطابق مع عدد الفئات في class_names")

In [ ]:
import os

file_path = '/content/drive/MyDrive/models/my_model.h5'
if os.path.exists(file_path):
    print("الملف موجود!")
else:
    print("الملف غير موجود. تحقق من المسار.")

In [ ]:
# إجراء التنبؤ
predictions = model.predict(img_array)

# استخراج الفئة المتنبأ بها ونسبة الثقة
predicted_class = class_names[np.argmax(predictions)]  # تحديد اسم الفئة المتنبأ بها
confidence = np.max(predictions) * 100  # حساب نسبة الدقة

# طباعة النتيجة
print(f'The picture is: {predicted_class}, Accuracy ratio: {confidence:.2f}%')